In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from xgboost import XGBRegressor
import xgboost as xgb
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import math
from sklearn.metrics import f1_score
from scipy import spatial

#Info : 552 subjects, 21 waves, 8378 rows
df = pd.read_csv('data/speed_dating_data.csv', encoding = "ISO-8859-1")


In [2]:
data2 = pd.concat([df.iloc[:,0], #ID
                  df.iloc[:,2], #gender
                  df.iloc[:,5],
                  df.iloc[:,9:12], #order of date
                  #df.iloc[:, 11:12], #partner's id
                  df.iloc[:, 12:13], #match, 
                  df.iloc[:,13:17], #int_corr, samerace, age of partner, race of partner 
                  df.iloc[:,17:23], #stated preferences
                  #df.iloc[:,23:24], #decision of partner - dec-o
                  df.iloc[:,24:28], #attr_o..fun_o
                  df.iloc[:,28:32],
                  df.iloc[:,24],   # rating by partner for 6 attributes, probability of matching with partner #amb-o has lots of missing values - try eliminating
                  #df.iloc[:,:35], # met, age, field
                  df.iloc[:,39:42],#race, importance of race, importance of religion
                  #df.iloc[:,42:43], #from   (DROPPED ZIPCODE AND INCOME AS THEY HAVE TOO MANY MISSING VALUES)
                  df.iloc[:,45:48], #goal, date, go_out
                  #df.iloc[:,48:50], #career, career_coded
                  df.iloc[:,50:67], #interests
                  #df.iloc[:,67:69], #exphappy, expnum (DROPPED - TOO MANY MISSING VALUES)
                  df.iloc[:,69:74],  #what subject looks for in the opposite sex - 6 attributes
                  df.iloc[:,74:75],  #shar1_1
                  df.iloc[:,81:87], #attr2_1,..shar2_1
                  df.iloc[:,87:92],  #attr3_1.. #91 is amb3_1 (how do you measure up)
                  #df.iloc[:,97:98], #decision of subject - dec
                  df.iloc[:,98:102], #ratings by subject for partner for the 6 attributes. 101 is fun
                  df.iloc[:,102:104], #amb,shar
                  df.iloc[:,104:107]],axis=1) #like, prob, met

data3 = data2.dropna()



In [5]:
wave=data3.wave.unique()
waves={}
for i in range(wave.shape[0]):
    waves[i]=data3[data3.wave==i]

In [6]:
waves[1].head()

,iid,gender,wave,order,partner,pid,match,int_corr,samerace,age_o,...,amb3_1,attr,sinc,intel,fun,amb,shar,like,prob,met
0,1,0,1,4,1,11.0,0,0.14,0,27.0,...,7.0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0
1,1,0,1,3,2,12.0,0,0.54,0,22.0,...,7.0,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0
3,1,0,1,5,4,14.0,1,0.61,0,23.0,...,7.0,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0
4,1,0,1,7,5,15.0,1,0.21,0,24.0,...,7.0,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0
5,1,0,1,6,6,16.0,0,0.25,0,25.0,...,7.0,4.0,9.0,7.0,4.0,6.0,4.0,6.0,5.0,2.0


In [8]:
drop_cols=['iid','id','pid','wave','partner','order']

In [9]:
sim=[]
flag=0
for k in waves:
    for l in waves:
        if k!=l:
            #print(k,l)
            for i in waves[k].iid.unique():
                for j in waves[l].iid.unique():
                    #print(i,j)

                    #print(i,j)
                    #change the below 2 lines
                    if ((data3.gender[data3.iid==i].all()==0 and data3.gender[data3.iid==j].all()==0) or (data3.gender[data3.iid==i].all()==1 and data3.gender[data3.iid==j].all()==1)):
                        per1=data3[data3.iid==i]
                        #print(per1.columns)
                        p1=per1.drop(drop_cols,axis=1)
                        per2=data3[data3.iid==j]
                        p2=per2.drop(drop_cols,axis=1)
                        d1=np.mean(p1)
                        d2=np.mean(p2)
                        #print(d1)
                        cos=spatial.distance.cosine(d1,d2)
                        #print(cos)
                        if cos>0.4:
                            flag=1
                            print('{} and {} are have similar preferences'.format(i,j,cos))
                            sim.append(list([i,j]))

                

if flag==0:
    print('No correlation')

20 and 52 are have similar preferences
20 and 66 are have similar preferences
20 and 106 are have similar preferences
20 and 108 are have similar preferences
20 and 163 are have similar preferences
20 and 216 are have similar preferences
20 and 222 are have similar preferences
6 and 241 are have similar preferences
8 and 241 are have similar preferences
20 and 286 are have similar preferences
20 and 315 are have similar preferences
20 and 415 are have similar preferences
52 and 20 are have similar preferences
32 and 57 are have similar preferences
32 and 79 are have similar preferences
32 and 83 are have similar preferences
32 and 90 are have similar preferences
32 and 174 are have similar preferences
32 and 252 are have similar preferences
32 and 262 are have similar preferences
52 and 373 are have similar preferences
52 and 400 are have similar preferences
66 and 20 are have similar preferences
57 and 32 are have similar preferences
57 and 241 are have similar preferences
64 and 241 

In [10]:
recomm=[]
for p in sim:
    a1=data3[data3.iid==p[0]]
    a2=data3[data3.iid==p[1]]
    p1=p[0]
    #print(a1)
    for i in range(a1.shape[0]):
        ab=a1.iloc[i]
        if ab.match==1:
            p2=int(ab['partner'])
            r=list([p[0],p2])            
            if r not in recomm:
                recomm.append(list([p[0],p2]))
                print('{} recommended to {} based on match of {}'.format(p2,p[0],p[1]))
    for i in range(a2.shape[0]):
        ab=a2.iloc[i]
        if ab.match==1:
            p2=int(ab['partner'])
            r=list([p[0],p2])
            if r not in recomm:
                recomm.append(list([p[1],p2]))
                print('{} recommended to {} based on match of {}'.format(p2,p[1],p[0]))


8 recommended to 20 based on match of 52
4 recommended to 108 based on match of 20
6 recommended to 108 based on match of 20
9 recommended to 108 based on match of 20
11 recommended to 108 based on match of 20
12 recommended to 163 based on match of 20
13 recommended to 163 based on match of 20
15 recommended to 163 based on match of 20
4 recommended to 6 based on match of 241
9 recommended to 6 based on match of 241
6 recommended to 241 based on match of 6
8 recommended to 241 based on match of 6
2 recommended to 8 based on match of 241
3 recommended to 8 based on match of 241
4 recommended to 8 based on match of 241
6 recommended to 8 based on match of 241
8 recommended to 8 based on match of 241
9 recommended to 8 based on match of 241
10 recommended to 8 based on match of 241
13 recommended to 315 based on match of 20
14 recommended to 315 based on match of 20
8 recommended to 20 based on match of 52
6 recommended to 57 based on match of 32
14 recommended to 79 based on match of 32

8 recommended to 20 based on match of 415
6 recommended to 241 based on match of 393
8 recommended to 241 based on match of 393
10 recommended to 373 based on match of 415
11 recommended to 373 based on match of 415
2 recommended to 492 based on match of 400
3 recommended to 492 based on match of 400
4 recommended to 492 based on match of 400
7 recommended to 492 based on match of 400
10 recommended to 492 based on match of 400
6 recommended to 241 based on match of 430
8 recommended to 241 based on match of 430
6 recommended to 71 based on match of 492
11 recommended to 373 based on match of 492
1 recommended to 400 based on match of 492
6 recommended to 400 based on match of 492


In [11]:
match_no=data3[data3.match==1]
match_num=match_no.match.count()
print('Number of matches in all the waves:{}'.format(match_num))
print('Number of additional recommendations:{}'.format(len(recomm)))

Number of matches in all the waves:981
Number of additional recommendations:223


In [12]:
print('Percent increase in probability of matches due to recommendation: ',(len(recomm)/match_num*100))

Percent increase in probability of matches due to recommendation:  22.73190621814475
